# Building the training dataset from video

On your Raspberry Pi, record a 20 second video showing the Evian bottle in various positions. Repeat the process for "coke" and "valser".

```
raspivid -o evian.h264 -t 20000
```

Now copy the H264 files from your Raspberry Pi to your DSVM, at path */data/drinks/raw*.

Now we use ffmpeg to extract a frame every 0.1s from each video, to generate a training set. Then we move 20% of the images from the training to the validation set.

In [33]:
%%bash
path=/data/drinks
rm -r $path/train $path/valid
validPercent=20
for dataset in coke evian valser; do
    mkdir -p  $path/train/$dataset $path/valid/$dataset
    ffmpeg -loglevel warning -i $path/raw/$dataset.h264 -vf fps=10 $path/train/$dataset/img%03d.jpg
    numfiles=$(ls -1q $path/train/$dataset/*.jpg | wc -l)
    num=$(expr $numfiles '*' $validPercent / 100)
    shuf -n $num -e $path/train/$dataset/*.jpg | xargs -i mv {} $path/valid/$dataset/
done



[h264 @ 0xa05100] Stream #0: not enough frames to estimate rate; consider increasing probesize
[swscaler @ 0xa22820] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1b1d820] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1860820] deprecated pixel format used, make sure you did set range correctly


Now we have under /data/drinks/train and /data/drinks/valid, training and validation datasets in the structure expected by Keras to generate batches.